<a href="https://colab.research.google.com/github/MatheoDaly/Death_Note/blob/master/Scrapping_And_Data_Cleaning_eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) ALL THE IMPORTS

In [0]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import numpy as np

# 2) GET THE WIKIPEDIA LINK AND THE NAME OF EVERY PERSONS

In [0]:
#Supprimer les balises
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr,'', raw_html)
  return cleantext

def fillContent(url_path):
  #Selectionne la page wikipédia et la met dans un objet content
  page = requests.get(url_path)
  contents = page.content

  #Séléctionne toutes les divs de class "mw-category-group" (div contenant les suicidés par lettre) et le met dans un tableau
  soup = BeautifulSoup(contents, 'html.parser')
  mydivs = soup.findAll("div", {"class": "mw-category-group"})

  #.replace(" ", "_")

  #prend le nom de tous les suicidés dans le tableau liste
  for i in mydivs:    
    myTitle = i.findAll('li')
    for i in myTitle:
      if not ('class="CategoryTreeSection"' in str(i)):
        listeName.append(cleanhtml(str(i)).replace(" ", "_"))
        listeLink.append("https://en.wikipedia.org/wiki/"+cleanhtml(str(i)).replace(" ", "_"))        

In [0]:
listeName = [] #liste contenant les noms
listeLink = [] #liste contenant les liens wikipédia
#fillContent('https://fr.wikipedia.org/wiki/Catégorie:Artiste_suicidé')

## All of the categories of suicide on wikipedia

In [9]:
listeDonnees=['https://en.wikipedia.org/w/index.php?title=Category:Male_suicides&pageuntil=Bystron%2C+David%0ADavid+Bystroň#mw-pages',
              'https://en.wikipedia.org/w/index.php?title=Category:Male_suicides&pagefrom=Bystron%2C+David%0ADavid+Bystroň#mw-pages',
              'https://en.wikipedia.org/w/index.php?title=Category:Male_suicides&pagefrom=Egan%2C+Richard%0ARichard+Egan+%28businessman%29#mw-pages',
              'https://en.wikipedia.org/w/index.php?title=Category:Male_suicides&pagefrom=Heggen%2C+Thomas%0AThomas+Heggen#mw-pages',
              'https://en.wikipedia.org/w/index.php?title=Category:Male_suicides&pagefrom=Lewis%2C+Lester%0ALester+Lewis#mw-pages',
              'https://en.wikipedia.org/w/index.php?title=Category:Male_suicides&pagefrom=Patterson%2C+Jeremy%0AJeremy+Patterson#mw-pages',
              'https://en.wikipedia.org/w/index.php?title=Category:Male_suicides&pagefrom=Sen+no%2C+Rikyu%0ASen+no+Rikyū#mw-pages',
              'https://en.wikipedia.org/w/index.php?title=Category:Male_suicides&pagefrom=Weidig%2C+Friedrich+Ludwig%0AFriedrich+Ludwig+Weidig#mw-pages',
              'https://en.wikipedia.org/wiki/Category:Female_suicides',
              'https://en.wikipedia.org/w/index.php?title=Category:Female_suicides&pagefrom=Mandel%2C+Miriam%0AMiriam+Mandel#mw-pages',
              ]
#listeDonnees=['https://fr.wikipedia.org/wiki/Catégorie:Actrice_suicidée']
for i in listeDonnees:
  fillContent(i)
  
len(listeLink)

1875

In [0]:
#scrapper toutes les pages dans notre listeLink
listeContent= []
for i in listeLink:
    pageSuicide = requests.get(i)
    pageSuicide.encoding = "utf-8"
    data = pageSuicide.text

    soup = BeautifulSoup(data, 'html.parser')
    #listeContent.append(cleanhtml(str(soup)).replace("\n", "").replace(":", ""))
    listeContent.append(str(soup).replace("\n", "").replace(":", ""))

# 3) DATA CLEANING

## Splitting the sentences into list of words

In [0]:
for i in range(len(listeContent)):
  listeContent[i]=listeContent[i].split()

## Selection of only the biography

In [0]:
ct = 0

cont = []
for i in range(len(listeContent)):
  flag = False
  cont2 = []
  for j in listeContent[i]:
    if(j=='id="Biographie">Biographie</span><span'):
      flag = True
    if(flag == True):
      cont2.append(j)
      #print(cont2)
      if('<h2' in j):
        flag = False
  cont.append(cont2)
# len(listeContent)

In [0]:
listeContent = cont

## Dataframe with biography

In [0]:
  
# Calling DataFrame constructor after zipping 
# both lists, with columns specified 
df = pd.DataFrame(list(zip(listeName, listeLink, listeContent)), columns =['Name', 'Link', 'Content']) 
#df = df.drop_duplicates(["Name"]) #supprimer les doublons

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3184 entries, 0 to 3183
Data columns (total 3 columns):
Name       3184 non-null object
Link       3184 non-null object
Content    3184 non-null object
dtypes: object(3)
memory usage: 74.8+ KB


In [0]:
for i in range(len(df['Content'])):
  #print(i+1)
  df['Content'][i] = " ".join(df['Content'][i])
  df['Content'][i] = cleanhtml(df['Content'][i])
  df['Content'][i] = df['Content'][i].split()

## Dataframe with only names and links

In [0]:
df = pd.DataFrame(list(zip(listeName, listeLink)), columns =['Name', 'Link']) 

In [11]:
df = df.drop_duplicates(["Name"]) #supprimer les doublons
df.head()

,Name,Link
0,Jack_Abbott_(author),https://en.wikipedia.org/wiki/Jack_Abbott_(aut...
1,Abdülaziz,https://en.wikipedia.org/wiki/Abdülaziz
2,Abu_Nidal,https://en.wikipedia.org/wiki/Abu_Nidal
3,Chris_Acland,https://en.wikipedia.org/wiki/Chris_Acland
4,Fort_Acuña,https://en.wikipedia.org/wiki/Fort_Acuña


## Saving the dataframe into a .csv file

In [0]:
from google.colab import files
df.to_csv('Suicides_wikipedia_scrapping.csv', sep=';', encoding='utf-8-sig', index=False)
files.download('Suicides_wikipedia_scrapping.csv')

# 4) Test of data visualisation

In [0]:
#on récupère chaque mot de la page

key=["pendaison","étrangl","strangul","suspend","tir ","balle","arme", "suffocation","suffoque","asphyxie","étouffement","étouff","étrangl", "immole","surdose","tire","défenestration","défenestr","overdose","barbituriques"]


p = 0
b = 0
i = 0
a = 0
d = 0
o = 0
hi = False
df["mort"] =np.nan
i = 0
for dfC in df["Content"]:
  for li in dfC.split():
    for k in key:
      if k in li:
        #print(dfC.split()[0], dfC.split()[1], " -> ", k)
        hi=True
        if k in ["pendaison","étrangl","strangul","suffocation","étouffement","étouff"]:
          p+=1
          df["mort"][i] = "pendaison"
        if k in ["tir","balle","tire","arme"]:
          b+=1
          df["mort"][i] = "par_balle"
        if k == "asphyxie":
          a+=1
          df["mort"][i] = "asphyxie"
        if k == "immole":
          i+=1
          df["mort"][i] = "immolation"
        if k in ["défenestration","défenestr"]:
          d+=1
          df["mort"][i] = "defenestration"
        if k in ["overdose","barbituriques"]:
          o+=1
          df["mort"][i] = "medicament"
  i+=1

#hi
 
morts_df = pd.get_dummies(df['mort'])
#df.drop('mort', axis=1, inplace=True)
df = df.join(morts_df)
df.head(15)

df[0].split()


In [0]:

df2 = pd.DataFrame({'lab':df["mort"].unique(), 'val':[sum(df["defenestration"]), sum(df["medicament"]), sum(df["par_balle"]), sum(df["asphyxie"]), 228, sum(df["pendaison"]), sum(df["immolation"])]})
ax = df2.plot.bar(x='lab', y='val', rot=15, title="Mort par ")

print("nan: ", len(df.index))
print("asphyxie: ", sum(df["asphyxie"]))
print("par_balle: ", sum(df["par_balle"]))
print("pendaison: ", sum(df["pendaison"]))
print("immolation: ", sum(df["immolation"]))
print("defenestration: ", sum(df["defenestration"]))
print("medicament: ", sum(df["medicament"]))

In [0]:
print("asphyxie: ", sum(df["asphyxie"]))
print("par_balle: ", sum(df["par_balle"]))
print("pendaison: ", sum(df["pendaison"]))
print("immolation: ", sum(df["immolation"]))
print("defenestration: ", sum(df["defenestration"]))
print("medicament: ", sum(df["medicament"]))

df['mort'].value_counts().plot(kind='bar')

#df["mort"].unique()